In [2]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# importing the dataset directly from huggingface
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ailsntua/QEvasion")

/Users/saiesh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/saiesh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent cal

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 3448
    })
    test: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 308
    })
})

In [5]:
print("\nTrain sample:\n", ds["train"][0])
print("\nColumns:\n", ds["train"].column_names)


Train sample:
 {'title': "The President's News Conference in Hanoi, Vietnam", 'date': 'September 10, 2023', 'president': 'Joseph R. Biden', 'url': 'https://www.presidency.ucsb.edu/documents/the-presidents-news-conference-hanoi-vietnam-0', 'question_order': 1, 'interview_question': 'Q. Of the Biden administration. And accused the United States of containing China while pushing for diplomatic talks.How would you respond to that? And do you think President Xi is being sincere about getting the relationship back on track as he bans Apple in China?', 'interview_answer': "Well, look, first of all, theI am sincere about getting the relationship right. And one of the things that is going on now is, China is beginning to change some of the rules of the game, in terms of trade and other issues.And so one of the things we talked about, for example, is that they're now talking about making sure that no Chineseno one in the Chinese Government can use a Western cell phone. Those kinds of things.And

In [6]:
MODEL = "microsoft/deberta-v3-base"

In [7]:
pip install sentencepiece protobuf

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

In [9]:
print("\nDataset Columns:", ds['train'].column_names)
print("Sample Entry:", ds['train'][0])


Dataset Columns: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label']
Sample Entry: {'title': "The President's News Conference in Hanoi, Vietnam", 'date': 'September 10, 2023', 'president': 'Joseph R. Biden', 'url': 'https://www.presidency.ucsb.edu/documents/the-presidents-news-conference-hanoi-vietnam-0', 'question_order': 1, 'interview_question': 'Q. Of the Biden administration. And accused the United States of containing China while pushing for diplomatic talks.How would you respond to that? And do you think President Xi is being sincere about getting the relationship back on track as he bans Apple in China?', 'interview_answer': "Well, look, first of all, theI am sincere about getting the relationship right. And one of the thing

In [10]:
LABEL_NORMALIZER = {
    # Clear Reply mappings
    'explicit': 'Explicit',
    'clear reply': 'Explicit',
    'clear': 'Explicit',
    
    # Ambivalent (Evasion) mappings
    'implicit': 'Implicit',
    'dodging': 'Dodging',
    'general': 'General',
    'deflection': 'Deflection',
    'partial': 'Partial/half-answer',
    'partial/half-answer': 'Partial/half-answer',
    
    # Clear Non-Reply mappings
    'declining': 'Declining to answer',
    'declining to answer': 'Declining to answer',
    'ignorance': 'Claims ignorance',
    'claims ignorance': 'Claims ignorance',
    'clarification': 'Clarification',
    
    # Fallback mappings
    'clear non-reply': 'Declining to answer',
    'ambivalent': 'Implicit', # Default ambivalent to Implicit if unspecified
    'ambiguous': 'Implicit'
}

In [11]:
raw_df = ds['train'].to_pandas()

expanded_rows = []

print(f"Processing {len(raw_df)} original rows...")

Processing 3448 original rows...


In [12]:
for _, row in raw_df.iterrows():
    q_text = row['interview_question']
    a_text = row['interview_answer']
    
    # Collect all potential labels for this row
    potential_labels = []
    
    # A. Check the 3 Annotators (Priority: High)
    # The FAQ states any annotator is valid, so we train on ALL of them[cite: 210].
    for col in ['annotator1', 'annotator2', 'annotator3']:
        val = row.get(col)
        if val and str(val).lower() not in ['nan', 'none', '']:
            potential_labels.append(str(val).strip())
            
    # B. If no annotators (Training set usually), use evasion_label
    if not potential_labels:
        val = row.get('evasion_label')
        if val and str(val).lower() not in ['nan', 'none', '']:
            potential_labels.append(str(val).strip())

    # C. Fallback: Use clarity_label if evasion is missing
    # (e.g., "Clear Reply" -> "Explicit")
    if not potential_labels:
        clarity = row.get('clarity_label')
        if clarity == 'Clear Reply':
            potential_labels.append('Explicit')
        elif clarity == 'Clear Non-Reply':
            potential_labels.append('Declining to answer') # Generic fallback

    # Add normalized rows to training data
    for raw_label in potential_labels:
        # Normalize (e.g., "Partial" -> "Partial/half-answer")
        clean_label = LABEL_NORMALIZER.get(raw_label, raw_label)
        
        # Only add if it's one of our valid target classes
        if clean_label in LABEL_NORMALIZER.values():
            expanded_rows.append({
                "text": q_text,
                "text_pair": a_text,
                "label_str": clean_label
            })

# Create final dataframe
df_train = pd.DataFrame(expanded_rows)
print(f"Final Training Size: {len(df_train)} (Exploded from {len(raw_df)})")
print(f"Classes Found: {df_train['label_str'].unique()}")

Final Training Size: 3448 (Exploded from 3448)
Classes Found: ['Explicit' 'General' 'Partial/half-answer' 'Dodging' 'Implicit'
 'Deflection' 'Declining to answer' 'Claims ignorance' 'Clarification']


In [13]:
unique_labels = sorted(df_train['label_str'].unique().tolist())
label2id = {l: i for i, l in enumerate(unique_labels)}
id2label = {i: l for i, l in enumerate(unique_labels)}

df_train['labels'] = df_train['label_str'].map(label2id)

# Compute Weights to handle imbalance (Explicit is ~30%, Clarification ~2%) [cite: 991]
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=np.unique(df_train['labels']), 
    y=df_train['labels']
)
weights_tensor = torch.tensor(class_weights, dtype=torch.float)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights_tensor = weights_tensor.to(device)

print(f"Class Weights applied: {weights_tensor}")

Class Weights applied: tensor([3.2194, 4.1643, 2.6421, 1.0055, 0.5427, 0.3642, 0.9925, 0.7851, 4.8495])


In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def preprocess_function(examples):
    return tokenizer(
        examples["text"], 
        examples["text_pair"], 
        truncation=True, 
        max_length=MAX_LENGTH,
        padding="max_length"
    )

hf_dataset = Dataset.from_pandas(df_train)
hf_dataset = hf_dataset.map(preprocess_function, batched=True)

# Prepare for PyTorch
cols = ['input_ids', 'attention_mask', 'labels']
if 'token_type_ids' in hf_dataset.column_names: cols.append('token_type_ids')
hf_dataset.set_format(type='torch', columns=cols)

# 90/10 Train/Validation Split
split_ds = hf_dataset.train_test_split(test_size=0.1, seed=42)

/Users/saiesh/Library/Python/3.9/lib/python/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


NameError: name 'Dataset' is not defined

In [ ]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Weighted Cross Entropy to help rare classes
        loss_fct = nn.CrossEntropyLoss(weight=weights_tensor)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Metric is Macro F1 as per shared task guidelines [cite: 1068]
    return {"f1_macro": f1_score(labels, predictions, average='macro')}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
).to(device)

training_args = TrainingArguments(
    output_dir="./clarity_final_model",
    learning_rate=2e-5,              # Recommended by paper [cite: 1162]
    per_device_train_batch_size=8,   # Adjust to 4 if OOM
    per_device_eval_batch_size=8,
    num_train_epochs=5,              # 5 epochs for convergence
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=split_ds["train"],
    eval_dataset=split_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\nStarting Training...")
trainer.train()

trainer.save_model("./final_clarity_submission_model")
tokenizer.save_pretrained("./final_clarity_submission_model")
print("Training Complete. Model saved to ./final_clarity_submission_model")

trainer.save_model("./final_clarity_submission_model")
tokenizer.save_pretrained("./final_clarity_submission_model")
print("Training Complete. Model saved to ./final_clarity_submission_model")

OSError: Can't load the model for 'microsoft/deberta-v3-base'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'microsoft/deberta-v3-base' is the correct path to a directory containing a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:
def map_prediction(pred_label):
    # Task 2 Output (Evasion Level) is just the label itself
    task2_output = pred_label
    
    # Task 1 Output (Clarity Level) - Based on Figure 3 Hierarchy
    if pred_label == 'Explicit':
        task1_output = 'Clear Reply'
        
    elif pred_label in ['Implicit', 'Dodging', 'General', 'Deflection', 'Partial/half-answer']:
        task1_output = 'Ambivalent Reply'
        
    elif pred_label in ['Declining to answer', 'Claims ignorance', 'Clarification']:
        task1_output = 'Clear Non-Reply'
        
    else:
        # Fallback (Should not happen if trained correctly)
        task1_output = 'Ambivalent Reply'
        
    return task1_output, task2_output